In [2]:
import pandas as pd


In [11]:
with open(r'C:\Users\imran\DataspellProjects\WalidCase\data\crunchbase_vs_predictions\kaggle_crunchbase_dataset.csv', 'r', encoding='utf-8', errors='ignore') as f:
    crunchbase = pd.read_csv(f)
crunchbase.rename(columns={' market ': 'market'}, inplace=True)
crunchbase = crunchbase[['name', 'market']]

startups = pd.read_csv(r"C:\Users\imran\DataspellProjects\WalidCase\data\raw\30k_startups_raw.csv", encoding='utf-8')

In [ ]:
crunchbase = crunchbase.notna()
startups = startups.notna()

# keep only rows where startup name is in crunchbase


present_in_updated_kws = crunchbase[crunchbase['name'].isin(startups['name'])]
present_in_updated_kws = present_in_updated_kws.merge(startups, on='name', how='left')


In [15]:
crunchbase = crunchbase[crunchbase['name'].isin(startups['name'])]
crunchbase = crunchbase.merge(startups, on='name', how='left')


In [17]:
crunchbase.dropna(inplace=True)
crunchbase.drop_duplicates(inplace=True)
crunchbase.drop(columns=['id'], inplace=True)

In [23]:
crunchbase['market'].value_counts()

 Software                  237
 Biotechnology             205
 Mobile                    126
 E-Commerce                106
 Curated Web                83
                          ... 
 Energy Efficiency           1
 Groceries                   1
 Unifed Communications       1
 Bio-Pharm                   1
 Billing                     1
Name: market, Length: 374, dtype: int64

In [31]:
# produce a dataframe where each market is on column 1 and the column 2 is a continuous string of all startup descriptions in that market called cb_description
crunchbase = crunchbase.groupby('market')['cb_description'].apply(lambda x: ' '.join(x)).reset_index()




In [33]:
crunchbase.rename(columns={'market': 'name'}, inplace=True)

In [55]:
import spacy
from tqdm import tqdm

class TextProcessing:
    def __init__(self, df: pd.DataFrame = None, industry=False, startup=False):
        self.nlp = spacy.load("en_core_web_sm")
        self.startups = pd.DataFrame([])
        self.industries = pd.DataFrame([])
        if startup:
            self.startups = df.copy()
        elif industry:
            self.industries = df.copy()
        else:
            raise ValueError("Please specify if the data is for startups or industries")

    def __iterate_rows(self):
        df = self.startups if not self.startups.empty else self.industries
        for index, row in tqdm(df.iterrows()):
            self.index = index
            if not self.industries.empty:
                self.about_us = row["keywords"]
            else:
                self.about_us = row["cb_description"]
            yield self

    def length_range(self, data, length_range=(30, 150)):

        self.startups = data.copy()
        self.startups.dropna(inplace=True)
        for i, row in self.startups.iterrows():
            length = len(row['cb_description'].split())
            if length < 15 or length > 60:
                self.startups.drop(i, inplace=True)

        return self.startups

    def remove_non_english_tokens(self, data=None):
        if data is not None:
            if not self.industries.empty:
                self.industries = data.copy()
            else:
                self.startups = data.copy()
        english_tokens = []
        for description in self.__iterate_rows():
            doc = self.nlp(self.about_us)
            tokens = [token.text for token in doc if token.lang_ == 'en' and token.is_alpha]
            self.about_us = " ".join(tokens)
            english_tokens.append(self.about_us)

        if not self.startups.empty:
            self.startups['cb_description'].replace(to_replace=self.startups['cb_description'].unique(),
                                                    value=english_tokens, inplace=True)
            return self.startups

        else:
            self.industries['keywords'].replace(to_replace=self.industries['keywords'].unique(), value=english_tokens,
                                                inplace=True)
            return self.industries

    def remove_noisy_tokens(self, data=None):
        if data is not None:
            if not self.industries.empty:
                self.industries = data.copy()
            else:
                self.startups = data.copy()

        cleaned_about_us = []
        for item in self.__iterate_rows():
            doc = self.nlp(self.about_us)
            for ent in doc.ents:
                if ent.label_:
                    self.about_us = self.about_us.replace(ent.text, "")
            cleaned_doc = self.nlp(self.about_us)

            tokens = [token.text.lower() for token in cleaned_doc if
                      not token.is_stop
                      and not token.is_punct
                      and not token.is_space
                      and not token.like_num
                      and not token.is_digit
                      and not token.is_currency
                      and not token.is_bracket
                      and not token.is_quote
                      and not token.is_left_punct
                      and not token.is_right_punct
                      and not token.like_url
                      and not token.like_email]

            self.about_us = " ".join(tokens)
            cleaned_about_us.append(self.about_us)
        if not self.industries.empty:
            self.industries['keywords'].replace(to_replace=self.industries['keywords'].unique(), value=cleaned_about_us,
                                                inplace=True)
            return self.industries
        else:
            self.startups['cb_description'].replace(to_replace=self.startups['cb_description'].unique(),
                                                    value=cleaned_about_us, inplace=True)
            return self.startups

    def lemma(self, data=None):
        if data is not None:
            if not self.industries.empty:
                self.industries = data.copy()
            else:
                self.startups = data.copy()
        lemmatized_about_us = []
        for description in self.__iterate_rows():
            doc = self.nlp(self.about_us)
            tokens = [token.lemma_ for token in doc]
            self.about_us = " ".join(tokens)
            lemmatized_about_us.append(" ".join(tokens))
        if not self.industries.empty:
            self.industries['keywords'].replace(to_replace=self.industries['keywords'].unique(),
                                                value=lemmatized_about_us, inplace=True)
            return self.industries
        else:
            self.startups['cb_description'].replace(to_replace=self.startups['cb_description'].unique(),
                                                    value=lemmatized_about_us, inplace=True)
            return self.startups

    @staticmethod
    def make_keywords_unique(df): # very ugly function but its okay
        unique_keywords = set()
        for index, row in df.iterrows():
            keywords = row['keywords'].split()
            unique_keywords.update(set(keywords))
        appended_keywords = []
        for index, row in df.iterrows():
            keywords = [keyword for keyword in row['keywords'].split() if keyword in unique_keywords and keyword not in appended_keywords]
            appended_keywords.extend(keywords)
            new_keys = ' '.join(keywords)
            df.at[index, 'keywords'] = new_keys
        return df




In [51]:
preprocess = TextProcessing(crunchbase, startup=True)


In [52]:
preprocess.remove_non_english_tokens()
preprocess.remove_noisy_tokens()
df = preprocess.lemma()

374it [00:46,  7.98it/s]
374it [01:08,  5.43it/s]
374it [00:20, 17.95it/s]


In [53]:
df.head(10)

,name,cb_description
0,3D,conceive found management team successful onli...
1,3D Printing,print usually outsource centrally produce corp...
2,Accounting,online brand guideline platform develop mainta...
3,Ad Targeting,boutique video network provide tailor channel ...
4,Advertising,trust partner create tailor solution improve u...
5,Advertising Exchanges,openx power highly relevant advertising global...
6,Advertising Networks,proprietary technology platform agency adverti...
7,Advertising Platforms,lead app engagement platform performance drive...
8,Aerospace,developer hybrid satellite cellular network sa...
9,Agriculture,early stage biotechnology company develop terp...


In [61]:
# remove words from the descriptions that are shared by any other description without using make_keywords_unique
df.rename(columns={'cb_description': 'keywords'}, inplace=True)
industry_df = df.copy()
preprocess = TextProcessing(industry_df, startup=False, industry=True)
preprocess.make_keywords_unique(industry_df)


,name,keywords
0,3D,conceive found management team successful onli...
1,3D Printing,print usually outsource centrally corporation ...
2,Accounting,guideline platform maintain integrity world br...
3,Ad Targeting,video network provide tailor channel optimisat...
4,Advertising,trust university enrollment marketing goal inn...
...,...,...
369,Wireless,harvester belgium slovenia enocean neul gearbo...
370,Women,innerwear
371,World Domination,
372,iPad,audanika harmonic harmonic audanika audanika h...


In [57]:
df.apply(lambda x: len(x['keywords'].split()), axis=1)

0       103
1       108
2        13
3       171
4      2003
       ... 
369     262
370       8
371      12
372      87
373     114
Length: 374, dtype: int64

In [58]:
industry_df.apply(lambda x: len(x['keywords'].split()), axis=1)

0       103
1       104
2         7
3       131
4      1255
       ... 
369      10
370       1
371       0
372       8
373       3
Length: 374, dtype: int64

In [76]:
for row in industry_df.iterrows():
    s = set(row[1]['keywords'].split())
    row[1]['keywords'] = ' '.join(s)

In [82]:
industry_df.apply(lambda x: len(x['keywords'].split()), axis=1)
x = industry_df[industry_df['keywords'].apply(lambda x: len(x.split())) > 30]
x

,name,keywords
0,3D,community tin affiliate aggregation fit intera...
1,3D Printing,engineer dream prototype needlessly realize ce...
3,Ad Targeting,provide improve content web transformative ret...
4,Advertising,pitch submit virtual training render conversio...
8,Aerospace,car install beamforming boat wind drone autono...
9,Agriculture,sodas ingredient worth aroma nutraceutical yea...
11,Analytics,interpret librarian simply page coca rapid rem...
12,Android,carpoole defend gamer payout fiddle term honey...
15,Apps,stake transfer storm detailed appoint cooperat...
19,Auctions,excess fee proceeding prefer pace having aucti...


In [81]:
from keybert import KeyBERT
model = KeyBERT()


In [88]:
for row in x.iterrows():
    doc = row[1]['keywords']
    keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words='english', use_mmr=True, diversity=0.3, use_maxsum=True, top_n=10)
    print({row[1]['name']: keywords})


{' Messaging ': [('decryption', 0.5319), ('transcribe', 0.4926), ('encrypt', 0.4604), ('telephone', 0.4092), ('kindle', 0.3699), ('transcranial', 0.3518), ('audios', 0.3449), ('landline', 0.3371), ('zillow', 0.2864), ('equips', 0.2838)]}
{' Mobile ': [('checklist', 0.3689), ('stripe', 0.3358), ('supercomputer', 0.3319), ('datacard', 0.3203), ('cryptos', 0.3112), ('telekom', 0.2799), ('offices', 0.2795), ('luxurious', 0.2655), ('overtraine', 0.2352), ('tagtagcity', 0.2294)]}
{' Music ': [('copyrightable', 0.5175), ('discographie', 0.4432), ('communities', 0.3509), ('netlab', 0.3441), ('videoclip', 0.3288), ('gig', 0.3262), ('chord', 0.3019), ('tabs', 0.2936), ('vinyl', 0.2917), ('outgoing', 0.2786)]}
{' Nanotechnology ': [('fullerene', 0.4292), ('ferrofluid', 0.3895), ('electrochemical', 0.3558), ('nanocompound', 0.3272), ('heterostructured', 0.3267), ('superparamagnetic', 0.3031), ('contaminant', 0.2293), ('envirox', 0.2267), ('phosphor', 0.1874), ('rejection', 0.1358)]}
{' Photography